In [1]:
import pytesseract
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import pandas as pd

In [2]:
def preprocessing_image(image):
    #image = cv2.imread('../raw_data/label_composition_images/IMG_1378.JPG')
    grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    noise = cv2.medianBlur(grayscale,5)
    #threshold = cv2.threshold(noise, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    #threshold doesn't really work

    #dilation
    kernel = np.ones((5,5),np.uint8)
    dilate = cv2.dilate(noise, kernel, iterations = 1)

    #erosion

    kernel = np.ones((5,5),np.uint8)
    erode= cv2.erode(dilate, kernel, iterations = 1)

    #opening - erosion followed by dilation
    #definition opening(image):
    #kernel = np.ones((5,5),np.uint8)
    #return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

    #canny edge detection

    #canny = cv2.Canny(threshold, 100, 200)

    #skew correction
    #definition deskew(image):
    coords = np.column_stack(np.where(erode > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = erode.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    deskew = cv2.warpAffine(erode, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)


    #template matching
    #definition match_template(image, template):
    #return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)

    #definition sharp(image):
    #sharp= cv2.bilateralFilter(deskew,9,75,75)
    return deskew

In [3]:
def image_to_string(image):
    
    preprocessed = preprocessing_image(image)
   
        #img = Image.fromarray(preprocessed, 'RGB')
    results = pytesseract.image_to_string(preprocessed)
    return results

In [4]:
image = cv2.imread('../raw_data/label_composition_images/IMG_1378.JPG')

In [5]:
ocr_result = image_to_string(image)
ocr_result

'  \n\nGB: Shell fabric1:100% polyester\nrecycled\n\nLining: 100% polyester recycled\nPadding: 100% recycled content\nNL: Bovenstoft: 100% polyester\ngerecyciede\n\nVoering: 100% polyester serecyelade\nVulling: 100% geracyclede inhoud\n\nDE: Oberstofft: 100% Polyester reeyveett\nFutter: 100% Polyester recyceit\nWattierung: 100% recycelt Inhalt\n\nFR: Etoffe extéricuret: 100%\npolyester recycle\n\nDoublura: 100% polyester recycle\nRemplissage: 100% recyclé content\n\n——_- —-\n\n \n\n \n\x0c'

In [6]:
ocr_splitted = ocr_result.splitlines()
ocr_splitted

['  ',
 '',
 'GB: Shell fabric1:100% polyester',
 'recycled',
 '',
 'Lining: 100% polyester recycled',
 'Padding: 100% recycled content',
 'NL: Bovenstoft: 100% polyester',
 'gerecyciede',
 '',
 'Voering: 100% polyester serecyelade',
 'Vulling: 100% geracyclede inhoud',
 '',
 'DE: Oberstofft: 100% Polyester reeyveett',
 'Futter: 100% Polyester recyceit',
 'Wattierung: 100% recycelt Inhalt',
 '',
 'FR: Etoffe extéricuret: 100%',
 'polyester recycle',
 '',
 'Doublura: 100% polyester recycle',
 'Remplissage: 100% recyclé content',
 '',
 '——_- —-',
 '',
 ' ',
 '',
 ' ',
 '']

In [7]:
fb3_keys = pd.read_csv('../eco_fashion_project/data/fibre_cleanedx3.csv')
fibres_list = fb3_keys['Material'].tolist()
fibres_list

['mechanically recycled nylon',
 'mechanically recycled polyester',
 'recycled nylon',
 'nylon recycled',
 'recycled polyester',
 'polyester recycled',
 'organic linen',
 'linen organic',
 'organic hemp',
 'hemp organic',
 'recycled cotton',
 'cotton recycled',
 'recycled wool',
 'wool recycled',
 'chemically recycled nylon',
 'chemically recycled polyester',
 'crailar',
 'in conversion cotton',
 'cotton in conversion',
 'monocel',
 'organic cotton',
 'cotton organic',
 'tencel',
 'linen',
 'hemp',
 'pla',
 'ramie',
 'modal lenzing viscose',
 'modal',
 'lenzing',
 'poly-acrylic',
 'polyester',
 'bamboo viscose',
 'cotton',
 'viscose',
 'rayon',
 'spandex',
 'elastane',
 'nylon',
 'wool',
 'acetate',
 'alpaca',
 'cashmere',
 'leather',
 'mohair',
 'bamboo',
 'organic wool',
 'wool organic',
 'silk',
 'banana',
 'cork',
 'jute',
 'nettle',
 'sisal',
 'spanish broom',
 'abaca',
 'crab shell',
 'lyocell',
 'coconut choir',
 'kapok',
 'ramie',
 'feathers',
 'feather',
 'orange fibre',
 'app

In [8]:
all_matches = []
for fibre in fibres_list:
    matching = [s for s in ocr_splitted if fibre in s]
    if matching:
        matched = [fibre, matching]
        all_matches.append(matched)
all_matches

[['polyester recycled', ['Lining: 100% polyester recycled']],
 ['polyester',
  ['GB: Shell fabric1:100% polyester',
   'Lining: 100% polyester recycled',
   'NL: Bovenstoft: 100% polyester',
   'Voering: 100% polyester serecyelade',
   'polyester recycle',
   'Doublura: 100% polyester recycle']]]

In [9]:
all_matches_df = pd.DataFrame(all_matches)
all_matches_df[0]

0    polyester recycled
1             polyester
Name: 0, dtype: object

In [10]:
max(all_matches_df[0], key=len)

'polyester recycled'

In [11]:
multi_fb_list = ['nylon', 'polyester', 'linen', 'hemp', 'cotton', 'wool', 'viscose', 'leather']

In [16]:
type(all_matches_df[0])

pandas.core.series.Series

In [17]:
all_matches_df[0].isin(['polyester'])

0    False
1     True
Name: 0, dtype: bool

In [18]:
#for fibre in multi_fb_list:
#    multi_matching = [s for s in all_matches_df[0] if fibre in s]

polyester_group = [s for s in fibres_list if "polyester" in s]
polyester_group

['mechanically recycled polyester',
 'recycled polyester',
 'polyester recycled',
 'chemically recycled polyester',
 'polyester']

In [23]:
all_matches_df[0].isin(polyester_group)

0    True
1    True
Name: 0, dtype: bool

In [25]:
all_matches_df[all_matches_df[0].isin(polyester_group)]

,0,1
0,polyester recycled,[Lining: 100% polyester recycled]
1,polyester,"[GB: Shell fabric1:100% polyester, Lining: 100..."


In [29]:
ind_all_matches_df = all_matches_df.set_index(0)
ind_all_matches_df

,1
0,
polyester recycled,[Lining: 100% polyester recycled]
polyester,"[GB: Shell fabric1:100% polyester, Lining: 100..."


In [57]:
ind_all_matches_df.loc['polyester recycled'][1]

['Lining: 100% polyester recycled']

In [58]:
ind_all_matches_df.loc[max(all_matches_df[0][all_matches_df[0].isin(polyester_group)], key=len)][1]

['Lining: 100% polyester recycled']

In [59]:
int_tags = ind_all_matches_df.loc[max(all_matches_df[0][all_matches_df[0].isin(polyester_group)], key=len)][1]
type(int_tags)

list

In [51]:
import re
percent = re.search(r'((\d+)[%])','Lining: 100% polyester recycled')
percent.group(0)

'100%'

In [60]:
percentage = re.search(r'((\d+)[%])',int_tags[0])
percentage.group(0)

'100%'

In [61]:
nylon_group = [s for s in fibres_list if "nylon" in s]
print(nylon_group)
linen_group = [s for s in fibres_list if "linen" in s]
print(linen_group)
hemp_group = [s for s in fibres_list if "hemp" in s]
print(hemp_group)
cotton_group = [s for s in fibres_list if "cotton" in s]
print(cotton_group)
wool_group = [s for s in fibres_list if "wool" in s]
print(wool_group)
viscose_group = [s for s in fibres_list if "viscose" in s]
print(viscose_group)
leather_group = [s for s in fibres_list if "leather" in s]
print(leather_group)

['mechanically recycled nylon', 'recycled nylon', 'nylon recycled', 'chemically recycled nylon', 'nylon']
['organic linen', 'linen organic', 'linen']
['organic hemp', 'hemp organic', 'hemp']
['recycled cotton', 'cotton recycled', 'in conversion cotton', 'cotton in conversion', 'organic cotton', 'cotton organic', 'cotton']
['recycled wool', 'wool recycled', 'wool', 'organic wool', 'wool organic']
['modal lenzing viscose', 'bamboo viscose', 'viscose']
['leather', 'apple leather', 'grape leather', 'pineapple leather', 'synthetic leather', 'leather synthetic']


In [63]:
multi_fb_group_list = [nylon_group, polyester_group, linen_group, hemp_group, cotton_group, wool_group, viscose_group, leather_group]

In [79]:
#for group in multi_fb_group_list:
print(all_matches_df[0])
print(nylon_group)
print(all_matches_df[0].isin(['mechanically recycled nylon', 'recycled nylon', 'nylon recycled', 'chemically recycled nylon', 'nylon']))
if all_matches_df[0].isin(nylon_group).any() == False:
    print("nothing will be done")
else:
    print("in nylon group")

0    polyester recycled
1             polyester
Name: 0, dtype: object
['mechanically recycled nylon', 'recycled nylon', 'nylon recycled', 'chemically recycled nylon', 'nylon']
0    False
1    False
Name: 0, dtype: bool
nothing will be done


In [81]:
if all_matches_df[0].isin(polyester_group).any() == True:
    fb_key = max(all_matches_df[0][all_matches_df[0].isin(polyester_group)], key=len)
    print(f"fiber: {fb_key}")
    int_tag_lines = ind_all_matches_df.loc[fb_key][1]
    print(f"interesting tag lines: {int_tag_lines}")
    pt = re.search(r'((\d+)[%])',int_tag_lines[0])
    pct = pt.group(0)
    print(f"percentage: {pct}")

fiber: polyester recycled
interesting tag lines: ['Lining: 100% polyester recycled']
percentage: 100%


In [82]:
if all_matches_df[0].isin(nylon_group).any() == True:
    fb_key = max(all_matches_df[0][all_matches_df[0].isin(nylon_group)], key=len)
    print(f"fiber: {fb_key}")
    int_tag_lines = ind_all_matches_df.loc[fb_key][1]
    print(f"interesting tag lines: {int_tag_lines}")
    pt = re.search(r'((\d+)[%])',int_tag_lines[0])
    pct = pt.group(0)
    print(f"percentage: {pct}")

In [88]:
tag_info = pd.DataFrame(columns = ["fiber", "percentage"])

for group in multi_fb_group_list:
    if all_matches_df[0].isin(group).any() == True:
        fb_key = max(all_matches_df[0][all_matches_df[0].isin(group)], key=len)
        print(f"fiber: {fb_key}")
        int_tag_lines = ind_all_matches_df.loc[fb_key][1]
        print(f"interesting tag lines: {int_tag_lines}")
        pt = re.search(r'((\d+)[%])',int_tag_lines[0])
        pct = pt.group(0)
        print(f"percentage: {pct}")
        tag_info.loc[len(tag_info)] = [fb_key,pct]
        #tag_info.append({'fiber': fb_key, 'percentage': pct}, ignore_index=True)

tag_info

fiber: polyester recycled
interesting tag lines: ['Lining: 100% polyester recycled']
percentage: 100%


,fiber,percentage
0,polyester recycled,100%


In [97]:
rest_group = fibres_list
for group in multi_fb_group_list:
    rest_group = [s for s in rest_group if s not in group]

rest_group

['crailar',
 'monocel',
 'tencel',
 'pla',
 'ramie',
 'modal',
 'lenzing',
 'poly-acrylic',
 'rayon',
 'spandex',
 'elastane',
 'acetate',
 'alpaca',
 'cashmere',
 'mohair',
 'bamboo',
 'silk',
 'banana',
 'cork',
 'jute',
 'nettle',
 'sisal',
 'spanish broom',
 'abaca',
 'crab shell',
 'lyocell',
 'coconut choir',
 'kapok',
 'ramie',
 'feathers',
 'feather',
 'orange fibre',
 'soy bean',
 'rubber',
 'angora',
 'casein',
 'elastothane',
 'polyamide',
 'polyethylene',
 'polytrimethylene',
 'teraphtalate']

In [98]:
if all_matches_df[0].isin(rest_group).any() == True:
    for match in all_matches_df[all_matches_df[0].isin(rest_group)]:
        fb_key = match[0]
        print(f"fiber: {fb_key}")
        int_tag_lines = match[1]
        print(f"interesting tag lines: {int_tag_lines}")
        pt = re.search(r'((\d+)[%])',int_tag_lines[0])
        pct = pt.group(0)
        print(f"percentage: {pct}")
        tag_info.loc[len(tag_info)] = [fb_key,pct]
tag_info

,fiber,percentage
0,polyester recycled,100%


# Test with WINNER :)

In [99]:
image2 = cv2.imread('../raw_data/label_composition_images/IMG_1388.JPG')

In [100]:
ocr_result2 = image_to_string(image2)
ocr_result2

"Aw’'1g 08/19\n\n36% Lana / Wool / Wolle / Laine /\nLana/ Ls\n\n35% Acrilica f Acrytie | Polyacryl /\nActylique / Actilicg / Aerflico\n\n18% Alpaca / Alpaca / Alpaka /\nAlpaga / Alnaca / Alpaca\n\n13% Poliamida / Nylon / Polyamid /\nFolyamide / Poliammidica / Polyamide\n\n1% Elastano / Elastane / Elasthan /\nElasthanne / Elastan / Elastano\n\nWAR ARE\n\nLAVAR AMANO - NO DEJAR OC\nNO USAR LEJIA BN REM\n\nNO UTILIZAR SECADORA\n\nPLANCKAR A TEMPERATURA BAJA\n\nMAX. 14%C\n\nNO LIMPIAR EN SECO\n\nLAVAR DEL REVES\n\nPLANCHAR DEL REVES\n\nSECAR =N PLANO\n\n—_=---\n\x0c"

In [101]:
ocr_splitted2 = ocr_result2.splitlines()
ocr_splitted2

["Aw’'1g 08/19",
 '',
 '36% Lana / Wool / Wolle / Laine /',
 'Lana/ Ls',
 '',
 '35% Acrilica f Acrytie | Polyacryl /',
 'Actylique / Actilicg / Aerflico',
 '',
 '18% Alpaca / Alpaca / Alpaka /',
 'Alpaga / Alnaca / Alpaca',
 '',
 '13% Poliamida / Nylon / Polyamid /',
 'Folyamide / Poliammidica / Polyamide',
 '',
 '1% Elastano / Elastane / Elasthan /',
 'Elasthanne / Elastan / Elastano',
 '',
 'WAR ARE',
 '',
 'LAVAR AMANO - NO DEJAR OC',
 'NO USAR LEJIA BN REM',
 '',
 'NO UTILIZAR SECADORA',
 '',
 'PLANCKAR A TEMPERATURA BAJA',
 '',
 'MAX. 14%C',
 '',
 'NO LIMPIAR EN SECO',
 '',
 'LAVAR DEL REVES',
 '',
 'PLANCHAR DEL REVES',
 '',
 'SECAR =N PLANO',
 '',
 '—_=---',
 '']

In [102]:
fb4_keys = pd.read_csv('../eco_fashion_project/data/fibre_cleanedx4.csv')
fibres_list = fb4_keys['Material'].tolist()
fibres_list

['mechanically recycled nylon',
 'mechanically recycled polyester',
 'recycled nylon',
 'nylon recycled',
 'recycled polyester',
 'polyester recycled',
 'organic linen',
 'linen organic',
 'organic hemp',
 'hemp organic',
 'recycled cotton',
 'cotton recycled',
 'recycled wool',
 'wool recycled',
 'chemically recycled nylon',
 'chemically recycled polyester',
 'crailar',
 'in conversion cotton',
 'cotton in conversion',
 'monocel',
 'organic cotton',
 'cotton organic',
 'tencel',
 'linen',
 'hemp',
 'pla polylactide',
 'ramie',
 'modal lenzing viscose',
 'modal',
 'lenzing',
 'acrylic',
 'polyacrylic',
 'polyester',
 'bamboo viscose',
 'cotton',
 'viscose',
 'rayon',
 'spandex',
 'elastane',
 'nylon',
 'wool',
 'acetate',
 'alpaca',
 'cashmere',
 'leather',
 'mohair',
 'bamboo',
 'organic wool',
 'wool organic',
 'silk',
 'banana',
 'cork',
 'jute',
 'nettle',
 'sisal',
 'spanish broom',
 'abaca',
 'crab shell',
 'lyocell',
 'coconut choir',
 'kapok',
 'ramie',
 'feathers',
 'feather',

In [103]:
len(fibres_list)

79

In [121]:
nylon_group = [s for s in fibres_list if "nylon" in s or "polyamide" in s]
print(nylon_group)
polyester_group = [s for s in fibres_list if "polyester" in s]
print(polyester_group)
linen_group = [s for s in fibres_list if "linen" in s]
print(linen_group)
hemp_group = [s for s in fibres_list if "hemp" in s]
print(hemp_group)
cotton_group = [s for s in fibres_list if "cotton" in s]
print(cotton_group)
wool_group = [s for s in fibres_list if "wool" in s]
print(wool_group)
viscose_group = [s for s in fibres_list if "viscose" in s]
print(viscose_group)
leather_group = [s for s in fibres_list if "leather" in s]
print(leather_group)

['mechanically recycled nylon', 'recycled nylon', 'nylon recycled', 'chemically recycled nylon', 'nylon', 'polyamide']
['mechanically recycled polyester', 'recycled polyester', 'polyester recycled', 'chemically recycled polyester', 'polyester']
['organic linen', 'linen organic', 'linen']
['organic hemp', 'hemp organic', 'hemp']
['recycled cotton', 'cotton recycled', 'in conversion cotton', 'cotton in conversion', 'organic cotton', 'cotton organic', 'cotton']
['recycled wool', 'wool recycled', 'wool', 'organic wool', 'wool organic']
['modal lenzing viscose', 'bamboo viscose', 'viscose']
['leather', 'apple leather', 'grape leather', 'pineapple leather', 'synthetic leather', 'leather synthetic']


In [123]:
multi_fb_group_list = [nylon_group, polyester_group, linen_group, hemp_group, cotton_group, wool_group, viscose_group, leather_group]

In [124]:
rest_group = fibres_list
for group in multi_fb_group_list:
    rest_group = [s for s in rest_group if s not in group]

print(rest_group)

['crailar', 'monocel', 'tencel', 'pla polylactide', 'ramie', 'modal', 'lenzing', 'acrylic', 'polyacrylic', 'rayon', 'spandex', 'elastane', 'acetate', 'alpaca', 'cashmere', 'mohair', 'bamboo', 'silk', 'banana', 'cork', 'jute', 'nettle', 'sisal', 'spanish broom', 'abaca', 'crab shell', 'lyocell', 'coconut choir', 'kapok', 'ramie', 'feathers', 'feather', 'orange fibre', 'soy bean', 'rubber', 'angora', 'casein', 'elastothane', 'polyethylene', 'polytrimethylene', 'teraphtalate']


In [125]:
nr_in_mfb_groups = []
for group in multi_fb_group_list:
    nr_in_mfb_groups.append(len(group))
print(nr_in_mfb_groups)
print(len(rest_group))
assert len(fibres_list) == sum(nr_in_mfb_groups, len(rest_group))

[6, 5, 3, 3, 7, 5, 3, 6]
41


In [146]:
def get_matches(ocr_splitted):
    all_matches = []
    for fibre in fibres_list:
        matching = [s.lower() for s in ocr_splitted if fibre in s.lower()]
        if matching:
            matched = [fibre, matching]
            all_matches.append(matched)

    all_matches_df = pd.DataFrame(all_matches)
    return all_matches_df

all_matches2_df = get_matches(ocr_splitted2)
all_matches2_df

,0,1
0,elastane,[1% elastano / elastane / elasthan /]
1,nylon,[13% poliamida / nylon / polyamid /]
2,wool,[36% lana / wool / wolle / laine /]
3,alpaca,"[18% alpaca / alpaca / alpaka /, alpaga / alna..."
4,polyamide,[folyamide / poliammidica / polyamide]


Findings:
polyamide should be disregarded (synonymous for nylon), 
acrylic not detected (since not exact wording)
alpaca % not correct (should be 15 instead of 18)
Try and take better picture

In [128]:
ind_all_matches2_df = all_matches2_df.set_index(0)
ind_all_matches2_df

,1
0,
elastane,[1% elastano / elastane / elasthan /]
nylon,[13% poliamida / nylon / polyamid /]
wool,[36% lana / wool / wolle / laine /]
alpaca,"[18% alpaca / alpaca / alpaka /, alpaga / alna..."
polyamide,[folyamide / poliammidica / polyamide]


In [145]:
def get_fiber_pct(df):
    ind_df = df.set_index(0)

    tag_info = pd.DataFrame(columns = ["fiber", "percentage"])

    for group in multi_fb_group_list:
        if df[0].isin(group).any() == True:
            grouped_fibres = df[0][df[0].isin(group)]

            fb_key = max(grouped_fibres, key=len)
            if fb_key == "polyamide" and grouped_fibres.iloc[0] == 'nylon':
                fb_key = 'nylon'
            
            pct = get_pct(ind_df, fb_key)
            tag_info.loc[len(tag_info)] = [fb_key,pct]

    if df[0].isin(rest_group).any() == True:
        for match in df[0][df[0].isin(rest_group)]:
            fb_key = match
            pct = get_pct(ind_df, fb_key)
            tag_info.loc[len(tag_info)] = [fb_key,pct]

    return tag_info

def get_pct(ind_df, fb_key):
    int_tag_lines = ind_df.loc[fb_key][1]
    #print(f"interesting tag lines: {int_tag_lines}")
    pt = re.search(r'((\d+)[%])',int_tag_lines[0])
    pct = pt.group(0)
    
    return pct

get_fiber_pct(all_matches2_df)

,fiber,percentage
0,nylon,13%
1,wool,36%
2,elastane,1%
3,alpaca,18%
